In [2]:
pip install transformers

     ---------------------------------------- 7.2/7.2 MB 262.3 kB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 1.0 MB/s eta 0:00:00
     -------------------------------------- 78.4/78.4 kB 624.9 kB/s eta 0:00:00
     -------------------------------------- 286.2/286.2 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 268.8/268.8 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 269.6/269.6 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 143.0/143.0 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:

pip install torch

     -------------------------------------- 162.6/162.6 MB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install Flask

Note: you may need to restart the kernel to use updated packages.


In [1]:
from flask import Flask, request, jsonify, session
from transformers import pipeline

# Initialize the Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Load the chatbot model
chatbot = pipeline("conversational")

# Admission-related FAQ (can be connected to backend later)
admission_info = {
    'procedures': 'The admission procedure involves submitting an application form, attending an interview, and meeting the eligibility criteria.',
    'requirements': 'Admission requires 12th-grade marksheet, entrance test results, and an identification document.',
    'deadlines': 'Application deadlines for the fall intake are July 31st, and for spring intake are November 30th.'
}

# Error response
default_response = "I'm sorry, I don't have that information at the moment. Please try asking something else."

# Function to get admission information
def get_admission_info(query):
    if 'procedure' in query.lower():
        return admission_info['procedures']
    elif 'requirement' in query.lower():
        return admission_info['requirements']
    elif 'deadline' in query.lower():
        return admission_info['deadlines']
    else:
        return default_response

# Save context between conversations
def remember_context(user_input):
    if 'previous_messages' not in session:
        session['previous_messages'] = []
    session['previous_messages'].append(user_input)

# Personalized response
def personalized_response():
    return f"Based on your previous questions, here’s more information: {session['previous_messages']}."

# Route to interact with the chatbot
@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    
    # Check if context exists (to remember previous conversations)
    remember_context(user_input)
    
    # Get the admission-related info
    admission_response = get_admission_info(user_input)
    
    # Call the chatbot model for a conversational reply (you can enhance this model to fine-tune for more specific responses)
    model_response = chatbot(user_input)[0]['generated_text']

    # Check if the chatbot can answer the query
    if admission_response != default_response:
        final_response = admission_response
    else:
        final_response = model_response

    # Return response in JSON
    return jsonify({
        'response': final_response,
        'context': personalized_response()
    })

# Error handling route
@app.route('/error', methods=['POST'])
def error():
    user_input = request.json.get('message')
    return jsonify({
        'response': "I'm unable to understand your query. Could you please rephrase?"
    })

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)


KeyError: "Unknown task conversational, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"